In [3]:
%run data.ipynb

In [4]:
import pandas as pd

# Reconciling Stripe

## Automatic reconciliation by merchantId

In [5]:
# copy df_strip to df_s
df_stripe_reconciliation = df_stripe.copy()

In [6]:
df_stripe_reconciliation.head()

,id,Created date (UTC),Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata)
0,ch_3OTXgHBYycQg1OMf0IfKN8Sw,2023-12-31 22:46:55,Monthly financial contribution to Logseq (Back...,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI7wZ2sLFYmMPi,https://opencollective.com/sanket-sharma,True,pm_1OTXmVBYycQg1OMfzXUwCvAm,pi_3OTXgHBYycQg1OMf0PkaNI3M,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/sanket-sharma,https://opencollective.com/logseq
3,ch_3OTXVMBYycQg1OMf0qVZ5tjm,2023-12-31 22:29:41,Financial contribution to OpenSCAD,Payment complete.,50.00,0.0,usd,50.00,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTXVpBYycQg1OMfien2j8Cr,pi_3OTXVMBYycQg1OMf02f4Vhyl,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/openscad
4,ch_3OTULSBYycQg1OMf10wlDceK,2023-12-31 19:06:47,Monthly financial contribution to LibreELEC,Payment complete.,10.00,0.0,usd,10.00,0.0,1.24,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,cus_PI4UZkic5DVDO1,https://opencollective.com/daniel-dolejska,True,pm_1OTULSBYycQg1OMf7Lt0BwEi,pi_3OTULSBYycQg1OMf1saV7WWG,0.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,https://opencollective.com/daniel-dolejska,https://opencollective.com/libreelec
5,ch_3OTUG8BYycQg1OMf0FRuZULZ,2023-12-31 19:03:18,Financial contribution to Spotube,Payment complete.,5.00,0.0,usd,5.00,0.0,0.77,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTUI5BYycQg1OMfAbkDmXhS,pi_3OTUG8BYycQg1OMf0AcGFa2r,0.25,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/spotube
7,ch_3OTT8EBYycQg1OMf0gR3PzrY,2023-12-31 17:49:54,Financial contribution to .fmbot (.fmbot lifet...,Payment complete.,49.99,0.0,usd,49.99,0.0,4.25,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OTT94BYycQg1OMf9AOHyAVx,pi_3OTT8EBYycQg1OMf0xdn31JV,2.50,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/fmbot


In [7]:
# df_platform where kind is CONTRIBUTION and paymentMethod.service is STRIPE
df_platform_stripe = df_platform[(df_platform['kind'] == 'CONTRIBUTION') & (df_platform['paymentMethod.service'] == 'STRIPE')]

In [8]:
df_platform_stripe.head()

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31 22:46:58.403000+00:00,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,f3fb4940-1026-4440-bd85-bc3c0cb439b0,f3fb4940-1026-4440-bd85-bc3c0cb439b0,CONTRIBUTION,NaN,NaN,NaN,9931763a-79b1-4141-923f-216db174f56e,CREDIT,Financial contribution to OpenSCAD,2023-12-31 22:29:44.179000+00:00,ch_3OTXVMBYycQg1OMf0qVZ5tjm,False,False,NaN,False,NaN,Credit,USD,5000,Amount,USD,5000,Amount,USD,4825,Amount,USD,4825,Amount,USD,-175,Amount,rmvrwng4-kj03dpbk-ljzpz57o-yl9e8xba,132610.0,OpenSCAD,openscad,False,COLLECTIVE,Collective,n4gx0bro-v5m96n0l-878qd8lk-3ey7jzwa,467683,Holland Hopson,guest-8761eb54,False,INDIVIDUAL,Individual,ywz9j4av-god8pg8g-8ly6mr35-nxklb0e7,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,e438b64b-473b-405b-be1c-67c7cfc58fba,e438b64b-473b-405b-be1c-67c7cfc58fba,CONTRIBUTION,NaN,NaN,NaN,91d79b1b-e38d-4559-be03-ad4a5f5fa301,CREDIT,Monthly financial contribution to LibreELEC,2023-12-31 19:06:49.150000+00:00,ch_3OTULSBYycQg1OMf10wlDceK,False,False,NaN,False,NaN,Credit,USD,1000,Amount,USD,1000,Amount,USD,926,Amount,USD,926,Amount,USD,-74,Amount,88rzownx-l9e50pxv-3vb6ymvb-dgk7j43a,204839.0,LibreELEC,libreelec,False,COLLECTIVE,Collective,e0adkjrv-8xwm69oy-zwl67z5n-4l93bgoy,744470,Daniel Dolejska,daniel-dolejska,False,INDIVIDUAL,Individual,eng0kzdy-vor4pzez-e7bpbma8-37xlw95j,True,True,True,TransactionPermissions,None,STRIPE,NaN,CREDITCARD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
55,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,8ee3d5b0-b031-4b06-b7ab-69a8419900c2,CONTRIBUTION,NaN,NaN,NaN,0870251c-c506-421d-b969-9b98f303bb90,CREDIT,Financial contribution to Spotube,2023-12-31 19:03:20.581000+00:00,ch_3OTUG8BYycQg1OMf0FRuZULZ,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,3kzxy4v0-7wlr6mvy-5m76mj9n-o8agdbe5,475166.0

In [9]:
# for each row in df_stripe, find the corresponding row in df_platform, match df_stripe['id'] with df_platform['merchantId']
# if the row exists, add the platform transaction id to df_stripe in a new column called platform_transaction_id
# if the row does not exist, set the platform_transaction_id to None

# Merge the two dataframes on the 'id' from df_stripe and 'merchantId' from df_platform
merged_df = pd.merge(df_stripe_reconciliation, df_platform[['merchantId', 'id']], left_on='id', right_on='merchantId', how='left', suffixes=('', '_platform_transaction'))

# Drop the extra 'merchantId' column from the merge
merged_df.drop(columns='merchantId', inplace=True)

# Assign the resulting dataframe back to df_stripe
df_stripe_reconciliation = merged_df

In [10]:
# count the number of Stripe transactions that have a id_platform_transaction
print(f"Stripe transactions that have a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])}")

# count the number of Stripe transactions that do not have a id_platform_transaction
print(f"Stripe transactions without a corresponding platform transaction: {len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()])}")

# print the percentage of Stripe transactions that have a corresponding platform transaction
print(f"Percentage of Stripe transactions that have a corresponding platform transaction: {round(len(df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].notnull()])/len(df_stripe_reconciliation)*100, 2)}%")

Stripe transactions that have a corresponding platform transaction: 131588
Stripe transactions without a corresponding platform transaction: 432
Percentage of Stripe transactions that have a corresponding platform transaction: 99.67%


## Reconciling remaining Stripe transactions

In [11]:
df_stripe_remainder = df_stripe_reconciliation[df_stripe_reconciliation['id_platform_transaction'].isnull()]

In [12]:
df_stripe_remainder.head()

,id,Created date (UTC),Description,Seller Message,Amount,Amount Refunded,Currency,Converted Amount,Converted Amount Refunded,Fee,Taxes On Fee,Converted Currency,Mode,Status,Statement Descriptor,Customer ID,Customer Description,Captured,Card ID,PaymentIntent ID,Application Fee,Application ID,from (metadata),to (metadata),id_platform_transaction
116,ch_3OSjH1BYycQg1OMf1SrSebC3,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfAPu0eks5,pi_3OSjH1BYycQg1OMf1hLSOPbZ,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
117,ch_3OSjH1BYycQg1OMf03ruobeK,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfOEXlem38,pi_3OSjH1BYycQg1OMf0l0hNQLu,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
118,ch_3OSjH1BYycQg1OMf0fkywX7k,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMftpAiBEfm,pi_3OSjH1BYycQg1OMf0QvSjaGu,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
119,ch_3OSjH0BYycQg1OMf04HatfLy,2023-12-29 16:51:06,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSjH3BYycQg1OMfqhqMfElP,pi_3OSjH0BYycQg1OMf0xkJN2s7,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN
175,ch_3OSN05BYycQg1OMf1xfVvxkZ,2023-12-28 17:04:15,Contribution to manjaro (US),Payment complete.,1.0,0.0,usd,1.0,0.0,0.33,0.0,usd,Live,Paid,OPEN SOURCE COLLECTIVE,NaN,NaN,True,pm_1OSN0EBYycQg1OMf6jMM1Jzf,pi_3OSN05BYycQg1OMf13FUXY3w,NaN,ca_68FQ4jN0XMVhxpnk6gAptwvx90S9VYXF,NaN,https://opencollective.com/manjaro-us,NaN


In [13]:
# count df_stripe_remainder by 'Status'
df_stripe_remainder.groupby(['Status']).size()

Status
Paid        143
Refunded    289
dtype: int64

In [14]:
# count df_stripe_remainder by 'Seller Message'
df_stripe_remainder.groupby(['Seller Message']).size()

Seller Message
One of your rules allowed this payment, overriding Stripe's risk evaluation and any address or CVC checks.     41
Payment complete.                                                                                             376
Stripe evaluated this payment as having elevated risk, and placed it in your manual review queue.              15
dtype: int64

In [15]:
# sum amount of df_stripe_remainder by status
df_stripe_remainder.groupby(['Status'])['Amount'].sum()

Status
Paid        3907.0
Refunded     293.0
Name: Amount, dtype: float64

In [16]:
# count df_stripe_remainder by 'Created date (UTC)' date only
# create a new column with the date only
df_stripe_remainder['Created date'] = pd.to_datetime(df_stripe_remainder['Created date (UTC)']).dt.date

df_stripe_remainder.groupby(['Created date']).size()

/tmp/ipykernel_2798063/2206905382.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_stripe_remainder['Created date'] = pd.to_datetime(df_stripe_remainder['Created date (UTC)']).dt.date


Created date
2023-01-25     1
2023-03-13    10
2023-03-14    15
2023-03-15     2
2023-03-16     7
              ..
2023-06-29     8
2023-06-30     2
2023-12-27     2
2023-12-28     5
2023-12-29     4
Length: 73, dtype: int64

In [17]:
df_stripe_remainder.to_csv(f'data/missing_osc_stripe_transactions.csv')

In [18]:
# count df_stripe_remainder by 'to (metadata)'
df_stripe_remainder.groupby(['to (metadata)']).size()

to (metadata)
https://opencollective.com/advanced-bim-frontend-developer-course      1
https://opencollective.com/curl                                       37
https://opencollective.com/hledger                                     1
https://opencollective.com/manjaro-us                                 10
https://opencollective.com/mvvmcross                                   1
https://opencollective.com/openaddresses                             324
dtype: int64

In [19]:
df_platform.head()

,id,uuid,kind,paymentMethod,payoutMethod,giftCardEmitterAccount,group,type,description,createdAt,merchantId,isRefunded,isRefund,refundTransaction,isOrderRejected,expense,__typename,amount.currency,amount.valueInCents,amount.__typename,amountInHostCurrency.currency,amountInHostCurrency.valueInCents,amountInHostCurrency.__typename,netAmount.currency,netAmount.valueInCents,netAmount.__typename,netAmountInHostCurrency.currency,netAmountInHostCurrency.valueInCents,netAmountInHostCurrency.__typename,paymentProcessorFee.currency,paymentProcessorFee.valueInCents,paymentProcessorFee.__typename,account.id,account.legacyId,account.name,account.slug,account.isIncognito,account.type,account.__typename,oppositeAccount.id,oppositeAccount.legacyId,oppositeAccount.name,oppositeAccount.slug,oppositeAccount.isIncognito,oppositeAccount.type,oppositeAccount.__typename,permissions.id,permissions.canRefund,permissions.canDownloadInvoice,permissions.canReject,permissions.__typename,paymentMethod.name,paymentMethod.service,paymentMethod.sourcePaymentMethod,paymentMethod.type,expense.id,expense.type,expense.description,expense.invoiceInfo,expense.tags,expense.payee.id,expense.payee.name,expense.payee.slug,expense.payee.type,expense.payee.__typename,expense.__typename,refundTransaction.id,payoutMethod.type,payoutMethod.name,giftCardEmitterAccount.id,giftCardEmitterAccount.name,giftCardEmitterAccount.slug,giftCardEmitterAccount.__typename
0,a985a0d8-c94e-4173-9918-3246e3231021,a985a0d8-c94e-4173-9918-3246e3231021,HOST_FEE_SHARE,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,DEBIT,Host Fee Share,2023-12-31 22:46:58.361000+00:00,None,False,False,NaN,False,NaN,Debit,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,-25,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3w-g860ej5l-avnodgx7,8686,Open Collective,opencollective,False,ORGANIZATION,Organization,vedj9wro-z3a56dzb-zllq7blg-8x4m0ykn,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,377540a1-80e4-45dd-8cf8-44e5a8d04f4d,377540a1-80e4-45dd-8cf8-44e5a8d04f4d,HOST_FEE,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Host Fee,2023-12-31 22:46:58.361000+00:00,None,False,False,NaN,False,NaN,Credit,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,50,Amount,USD,0,Amount,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004.0,Open Source Collective,opensource,False,ORGANIZATION,Organization,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914,Logseq,logseq,False,COLLECTIVE,Collective,vrgbk357-l4x96ey9-yl4pomew-a0jdyzn8,True,True,True,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,d8b98e5b-60a2-4bb0-8e66-a0528ecaaf7c,d8b98e5b-60a2-4bb0-8e66-a0528ecaaf7c,HOST_FEE,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,DEBIT,Host Fee,2023-12-31 22:46:58.352000+00:00,None,False,False,NaN,False,NaN,Debit,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,-50,Amount,USD,0,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,8a47byg9-nxozdp80-xm6mjlv0-3rek5w8k,11004,Open Source Collective,opensource,False,ORGANIZATION,Organization,a47byg9n-xozdp8n7-nlmpmjlv-03rek5w8,False,True,False,TransactionPermissions,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,20e9111c-db7b-4002-8dc1-1c2538ffd6b2,CONTRIBUTION,NaN,NaN,NaN,00e1e29f-3348-4ebd-9598-c080adc4cbaf,CREDIT,Monthly financial contribution to Logseq (Back...,2023-12-31 22:46:58.403000+00:00,ch_3OTXgHBYycQg1OMf0IfKN8Sw,False,False,NaN,False,NaN,Credit,USD,500,Amount,USD,500,Amount,USD,448,Amount,USD,448,Amount,USD,-52,Amount,lk9mbw7y-48r3zq3m-gddp0ej5-lavnodgx,166914.0,Logseq,logseq,False,COLLECTIVE,Collective,n4gx0bro-v5m96nkd-3ay6d8lk-3ey7jzwa,744511,Sanket Sharma,sanket-sharma,False,INDIVIDUAL,Individual,8k03reyd-5agmq5r7-rlyplbwo-z7j4nxv9